In [3]:
def  pivot(N, B, A, b, c, v, l, e):
    '''
    N对应非基本元变量下标,B对应基本元变量下标,A对应非基本元在约束方程组中的系数相反数形成的矩阵,
    b对应约束条件中小于等于号右边的数值集合,c对应目标函数中变量系数形成的集合,v对应当前目标函数的取值,
    l对应转出变量下标在B中的位置，e对应转入变量下标在N中的位置
    '''
    b[l] = b[l] / A[l][e]  #将给定等式的常数和除了传入变量意外的其他变量对应的系数除以转入变量对应的系数
    for j in range(len(A[0])):
        if j == e:
            continue
        A[l][j] = A[l][j] / A[l][e]
    A[l][e] = 1 / A[l][e]  #这是等号左边变量转入右边后对应的系数
    for i in range(len(A)):  #改变其他等式常数和变量系数
        if i == l:
            continue
        b[i] = b[i] - A[i][e] * b[l]
        for j in range(len(A[0])): 
            if j == e:
                continue
            A[i][j] = A[i][j] - A[i][e] * A[l][j]
        A[i][e] = -A[i][e] * A[l][e]  #由于A中元素对应约束条件中变量系数的相反数,因此A中元素相乘后符号会负负得正，因此前面要加个负号
    v = v + c[e] * b[l]  #变量替换后目标修改目标函数的值
    for j in range(len(c)):  #目标函数中转出变量替换成转入变量后修改相关xishu7
        if j == e:
            continue
        c[j] = c[j] - c[e] * A[l][j]
    c[e] =  -c[e] * A[l][e]
    leaving_variable_index = B[l]
    entering_variable_index = N[e]
    B[l] = entering_variable_index
    N[e] = leaving_variable_index
    return  (N, B, A, b, c, v)
A = [
    [1, 1, 3],
    [2, 2, 5],
    [4, 1, 2]
    ]
B = [4, 5, 6]
N = [1, 2, 3]
b = [30, 24, 36]
c = [3, 1, 2]
v = 0
l = 2 
e = 0
(N, B, A, b, c, v)=pivot(N, B, A, b, c,v, l, e)
print("index of no basic variables after pivoting: ", N)
print("index of basic variables after pivoting: ", B)
print("coefficients of no basic variables after pivoting: ", A)
print("object function coefficients after pivoting is : ", c)
print("object function value after pivoting is : ", v)

index of no basic variables after pivoting:  [6, 2, 3]
index of basic variables after pivoting:  [4, 5, 1]
coefficients of no basic variables after pivoting:  [[-0.25, 0.75, 2.5], [-0.5, 1.5, 4.0], [0.25, 0.25, 0.5]]
object function coefficients after pivoting is :  [-0.75, 0.25, 0.5]
object function value after pivoting is :  27.0


In [7]:
import sys
def  simplex(N, B, A, b, c, v):
    mini_increase = sys.maxsize
    l = 0
    for e in range(len(c)): #在目标函数中找到系数大于0的对应变量
        if c[e] <= 0:
            continue
        for i in range(len(A)): #看哪个约束条件能让给定变量的增加值最小
            if  A[i][e] > 0 and b[i] / A[i][e] < mini_increase:
                mini_increase = b[i] / A[i][e]
                l = i
        if mini_increase == sys.maxsize: #目标函数没有最优解,它可以无限制增长
            return False
        (N, B, A, b, c, v) = pivot(N, B, A, b, c, v, l, e)
    for i in range(len(B)):
        print("x{0} : {1}".format(B[i], b[i]))
    for i in range(len(N)):
        print("x{0} : 0".format(N[i]))
    print("optimal value is : " , v)
    return (N, B, A, b, c, v)
A = [
    [1, 1, 3],
    [2, 2, 5],
    [4, 1, 2]
    ]
B = [4, 5, 6]
N = [1, 2, 3]
b = [30, 24, 36]
c = [3, 1, 2]
v = 0
simplex(N, B, A, b, c, v)

x4 : 18.0
x2 : 4.0
x1 : 8.0
x6 : 0
x5 : 0
x3 : 0
optimal value is :  28.0


([6, 5, 3],
 [4, 2, 1],
 [[0.0, -0.5, 0.5],
  [-0.3333333333333333, 0.6666666666666666, 2.6666666666666665],
  [0.3333333333333333, -0.16666666666666666, -0.16666666666666663]],
 [18.0, 4.0, 8.0],
 [-0.6666666666666666, -0.16666666666666666, -0.16666666666666663],
 28.0)

In [8]:
def  is_system_feasible(A, b, c):
    has_to_check = False
    for k in range(len(b)): #先检验是否存在小于0的常数
        if b[k] < 0:
            has_to_check = True
            break
    index = 1
    N = []
    B = []
    v = 0
    for i in range(len(A[0])): #设置非基本远变量的下标
        N.append(index)
        index += 1
    for i in range(len(A)): #设置基本远的下标
        B.append(index)
        index += 1
    if has_to_check is False:  #系统存在最优解,返回它的标准形式
        return True
    N_copy = N.copy()
    N_copy.insert(0, 0)  #加入新变量x0的下标
    B_copy = B.copy()
    b_copy = b.copy()
    c_copy = [-1, 0, 0] #目标函数只有一个参数-x0
    A_copy = A.copy()
    for i in range(len(A)):
        A_copy[i].insert(0, -1)  #每个约束条件都添加x0
    l = k
    (N_copy, B_copy, A_copy, b_copy, c_copy, v) = pivot(N_copy, B_copy, A_copy, b_copy, c_copy, v, l, 0)  #转换目标函数使得它包含有系数为正的变量
    (N_copy, B_copy, A_copy, b_copy, c_copy, v) = simplex(N_copy, B_copy, A_copy, b_copy, c_copy, v)
    for i in range(len(B_copy)):
        if B_copy[i] == 0 and b[i] != 0:
            return False
    return True

In [9]:
A=[[2, -1], [1, 5]]
c = [2, -1]
b = [2, -4]
v = 0
res = is_system_feasible(A, b, c)
if res is True:
    print("the given system is feasible")
else:
    print("the given system is infeasible")

x3 : 6.0
x0 : 4.0
x4 : 0
x1 : 0
x2 : 0
optimal value is :  -4.0
the given system is infeasible
